# Uebung 3 Merkmalsextraktion, binaere Klassifikation

Das Ziel dieser Übung ist die Integration der erlernten **Merkmalsextraktionsverfahren** in eine Klassifikationspipeline. Die einzelnen Schritte sind in der folgenden Abbildung veranschaulicht.

Dabei werdet ihr euch zunächst einmal der Unterscheidung von jeweils **zwei Klassen** widmen, also binären Klassifikationsproblemen. Nachdem ihr alle Aufgaben gelöst habt, könnt ihr die Anwendung für alle Klassen erweitern.

Das primäre Ziel ist der Aufbau einer **funktionierenden** Klassifikationspipeline und die Evaluation der Ergebnisse. Fokussiert euch zunächst auf die Implementierung. Wenn ihr alle Schritte implementiert und die Ergebnisse evaluiert habt, könnt ihr euch Gedanken über die Optimierung der Ergebnisse machen. 

Ihr werdet einen Subset aus dem **Industrial 100** Datensatz verwenden. Der Subset enthält die ersten 15 Klassen des Datensatzes. Eine Übersicht über alle ClassIDs und deren Bezeichnungen findet ihr in der csv-Datei **Industrial100-labels.csv**.


Den Datensatz könnt ihr von der tubCloud herunterladen und in den BGA2-Ordner ablegen (oder auf eurem eigenen Rechner). Anbei der **Link zum Datensatz**: https://tubcloud.tu-berlin.de/s/7ZRADfF4kdJGSma

**WICHTIG:** Die Daten aus diesem Subset solltet ihr nicht für die Hausaufgabe verwenden!




<img src="./ipynb_bilder/klassifikationspipeline.png"  />

## Imports

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog, sift
from skimage import exposure

from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.model_selection import learning_curve, train_test_split, ShuffleSplit
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import recall_score, precision_score, precision_recall_curve

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import interact, interactive, fixed, interact_manual, widgets

## Datenexploration
1. Schaut euch die ersten fünfzehn Klassen des Datensatzes "Industrial 100" an. Nutzt dafür euer Vorwissen (optional auch Hilfsfunktionen) aus Übung 1. Wählt für euch eine geeignete Methode (z.B. interaktive Anzeige mit ipywidgets.interact oder pyplot.subplots), um die 15 Klassen mitsamt ClassIDs und deren Bezeichnungen zu visualisieren. Ein Bild pro Klasse ist ausreichend. 
2. Wählt zwei Klassen aus, mit denen ihr die Klassifikationspipeline aufbaut. Dies könnt ihr im Laufe der Übung variieren, um schwierigere und leichtere Klassifikationsprobleme zu untersuchen, je nachdem ob die Klassen sehr ähnlich sind oder sich sehr unterscheiden. 

## Merkmalsextraktion
1. Extrahiert die [HOG](https://scikit-image.org/docs/0.20.x/auto_examples/features_detection/plot_hog.html#sphx-glr-auto-examples-features-detection-plot-hog-py)-Features für die von euch ausgewählten Klassen. Erweitert dafür euer Code aus Übung 2 so, dass ihr Features aus allen Bildern der beiden Klassen extrahiert.  
2. Unterteilt die Daten in Trainings- und Validierungsdaten. Ein mögliches Verhältnis wäre z.B. 75% Trainingsdaten und 25% Validierungsdaten. Dafür könnt ihr beispielsweise die [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)-Funktion aus der scikit-learn-Bibliothek nutzen. 
3. Nachdem ihr alle Schritte der Klassifikationspipeline für HOG-Features implementiert habt, könnt ihr zu dieser Teilaufgabe zurückkehren und die SIFT-Features extrahieren und die Klassifikationspipeline mit SIFT-Features aufbauen.

### Hinwese

**Zu 1.** 
Um die Arbeitsspeicherprobleme zu vermeiden, legt euch eine Liste mit den Bildpfaden der jeweiligen Ordner an. Für Merkmalsextraktion könnt ihr eine for-Schleife nutzen, um die Bilder einzeln einzulesen. 

**WICHTIG**: Bevor ihr Merkmale extrahiert, sollt ihr Bilder skalieren (z.B. mit resize), so dass sie gleich groß sind.

Speichert die extrahierten Merkmalsvektoren, damit sie diese nicht jedes Mal generieren müsst (siehe beigefügte Code Snippets)

**Zu 2.** 
Um die train_test_split-Funktion zu benutzen, solltet ihr zusätzlicht zu den Merkmalsvektoren, die ihr generiert habt, die Label- (ClassID-)Vektoren anlegen. 

Überprüft nach dem Nutzen der train_test_split-Funktion, dass alle Vektorgrößen übereinstimmen (siehe beigefügte Code Snippets)

Beispiel (bezieht sich nicht auf den Übungsdatensatz):


Shape of 
- X_train_img: (1800, 1568) (1800 - Anzahl der Merkmalsvektoren, 1568 - Länge des Merkmalsvektors)
- X_test_img: (600, 1568) 
- y_train_img: (1800,) 
- y_test_img: (600,)

### Code Snippets

In [ ]:
# save extracted features as numpy array
# to do: change the variable names according to your code
with open('hog_features_labels_all_classes.npy', 'wb') as f:
    np.save(f, np.array(hog_featutes_list))
    np.save(f, np.array(class_labels_list))
    np.save(f, np.array(img_paths))
    

In [ ]:
# load saved variables
with open('hog_features_labels_all_classes.npy', 'rb') as f:
    a = np.load(f)
    b = np.load(f)
    c = np.load(f)

In [ ]:
# print the shape of train and teat arrays after usage of train_test_split-function
# to do: change the variable names according to your code
print('Shape of \nX_train_img: {} \
\nX_test_img: {} \ny_train_img: {} \
\ny_test_img: {}'.format(X_train_img.shape, \
X_test_img.shape, y_train_img.shape, y_test_img.shape))

# the code does not work for lists only for numpy arrays
# see the following link to get the shape of a list:

# https://sparkbyexamples.com/python/get-shape-of-list-in-python/
# num_rows list
len(my_list)
# num_columns list
len(my_list[0])
print(np.shape(my_list)

## PCA
Führt eine PCA auf den Daten aus und projiziert die Daten dann in den neuen Merkmalsraum. Das gibt euch Aufschluss darüber, wie schwierig euer Klassifikationsproblem ist beziehungsweise wie gut eure Features geeignet sind, um die Klassen zu unterscheiden. 

Unter folgendem [Link](https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_vs_lda.html#sphx-glr-auto-examples-decomposition-plot-pca-vs-lda-py) findet ihr ein Beispiel (2D PCA für den Iris-Datensatz)

Nachdem ihr alle Schritte der Klassifikationspipeline implementiert und eure Ergebnisse evaluiert habt, könnt ihr die Klassifikation mit den durch die PCA reduzierten Feature-Vektoren durchführen und die Ergebnisse mit den Ergebnissen ohne PCA vergleichen. 
Konntet ihr die Unterschiede feststellen?

### Hinweise 
PCA - theoretische Grundlagen und Implementierung in Python einfach erklärt:
- https://www.youtube.com/watch?v=FgakZw6K1QQ
- https://www.youtube.com/watch?v=Lsue2gEM9D0


### Code Snippet
#If 0 < n_components < 1 and svd_solver == 'full', 
#select the number of components such that the amount of variance 
#that needs to be explained is greater than the percentage specified by n_components.


- pca_70 = PCA(n_components=.70, svd_solver='full')
- pca_70.fit(feature_data_hog)
- pca_70.n_components_
- pca_70.explained_variance_ratio_

In [ ]:
#Anbei ein Codesnippet für die Berechung und für die Visualisierung der PCA für die ersten zwei Hauptkomponenten 
# TO DO: definiere X- und y-Variablen


pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)
plt.xlabel('first principal component')
plt.ylabel('second principal component')
plt.title('Plot 1')
plt.scatter(X2D[:, 0], X2D[:, 1],c=y, cmap=plt.cm.nipy_spectral,edgecolor='k')
plt.show()

## Klassifikation
Für diese Teilaufgabe könnt ihr [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) Klassifikator, 
 [SVM](https://scikit-learn.org/stable/modules/svm.html)-Klassifikator oder [MLP](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)-Klassifikator der scikit-learn-Bibliothek ausprobieren.

In [ ]:
# Anwendungsbeispiel für Naive Bayes Klassifikator
gnb = GaussianNB()
# TO DO: Passe die Input- und Output-Parameter entsprechend den von dir gewählten Variablennamen an 
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

## Evaluation
Zur Beurteilung der Klassifikationsleistung des Klassifikators könnt ihr die CCR auf den Validierungsdaten berechnen und euch die Konfusionsmatrix anschauen. Dafür könnt ihr beispielsweise die [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)-Funktion, die [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion#sklearn.metrics.confusion_matrix)-Funktion, [f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)-Funktion der scikit-learn-Bibliothek nutzen. Experimentiert ein wenig und vergleicht die Ergebnisse für verschiedene Klassen und Konfigurationen eures Klassifikators.


Optional: Erstellung der Lernkurve
Das Ziel ist, zu schauen, wie sich die Fehler auf den Trainings- und Validierungsdaten mit der Menge der genutzten Trainingsdaten verändern. Dies wird Aufschluss darüber geben, wie ihr eure Klassifikationsleistung am effektivsten verbessern könnt. Man kann daraus ablesen, ob z.B. die Features ungeeignet sind oder das Modell des Klassifikators zu simpel ist, oder ob z.B. mehr Trainingsdaten helfen würden um die Klassifikationsleistung zu verbessern. Zur Erstellung der Lernkurve könnt ihr die [learning_curve](https://scikit-learn.org/stable/modules/learning_curve.html#learning-curve)-Funktion der scikit-learn-Bibliothek nutzen.

Nach dem Bearbeiten dieser Aufgabe sollt ihr folgende Fehlermaße / Gütekriterien definieren können und erklären, wofür sie verwendet werden: 
- accuracy score
- confusion matrix
- f1 score
- true positives
- true negatives
- false positives
- false negatives
- precision
- recall

### Accuracy

In [ ]:
# TO DO: Passe die Input- und Output-Parameter entsprechend den von dir gewählten Variablennamen an 
accuracy_score = accuracy_score(y_test, y_pred)
print('Accuracy score for Classes X & Y is: {}'.format(accuracy_score))

### Confusion Matrix

In [ ]:
# TO DO: Passe die Input- und Output-Parameter entsprechend den von dir gewählten Variablennamen an 

cm = confusion_matrix(y_test, y_pred)

cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
# TO DO: Passe die Input- und Output-Parameter entsprechend den von dir gewählten Variablennamen an 

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)

### F1-Score
Schaut euch die [scikit-learn-Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) an, um die Ergebnisse zu interpretieren. Achtet dabei auf die Implementierung des **average**-Parameters.

In [ ]:
# TO DO: Passe die Input- und Output-Parameter entsprechend den von dir gewählten Variablennamen an 
# Untersuche die Unterschiede für average='macro', average='micro', average='weighted', average=None
f1_score(y_test, y_pred, average='macro')

### Precision / Recall

In [ ]:
# TO DO: Passe die Input- und Output-Parameter entsprechend den von dir gewählten Variablennamen an 
# Untersuche die Unterschiede für average='macro', average='micro', average='weighted', average=None
# Link zur scikit-learn Doku: 
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html
precision_score(y_test, y_pred, average='macro')

In [ ]:
# Link zur scikit-learn Doku: 
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
recall_score(y_test, y_pred, average='macro')